# 📌 Adım 4 – Ortalama Bir CNN Yaz, TorchScript’e Export Et ve C++’ta Çalıştır

Bu adımda:

1. Python + PyTorch ile basit ama gerçekçi bir **CNN modeli** yazıyorum.  
2. Modeli TorchScript formatına çevirip `model_ts.pt` olarak kaydediyorum.  
3. C++ tarafında bu modeli, OpenCV’den aldığım `tensor_image` üzerine çalıştırıyorum.

Amaç:  
LibTorch + OpenCV pipeline’ını **gerçek bir görüntü modeli** ile uçtan uca görmek.

---

## 🔹 4.1 – Python Tarafı: SimpleCNN + TorchScript Export

Önce Python tarafında basit bir CNN tanımlıyorum:

- Giriş: `[1, 3, 64, 64]` (batch=1, RGB, 64×64)
- Çıkış: `10` sınıf (örnek: 10 kategori)

Aşağıdaki kodu `export_cnn_ts.py` gibi bir dosyaya kaydedip çalıştırıyorum:

```python
# SORU:
#  - 3x64x64 bir görüntü için ortalama bir CNN modeli nasıl tanımlarım?
#  - Bu modeli TorchScript'e çevirip C++ tarafında kullanmak için nasıl kaydederim?

import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes: int = 10):
        super().__init__()
        # [B, 3, 64, 64]
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)   # [B,16,64,64]
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # [B,32,64,64]
        self.pool = nn.MaxPool2d(2, 2)                            # boyut yarıya iner
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # [B,64,16,16] civarı
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))               # [B,64,4,4]
        self.fc = nn.Linear(64 * 4 * 4, num_classes)              # [B,10]

    def forward(self, x):
        x = F.relu(self.conv1(x))     # [B,16,64,64]
        x = self.pool(x)              # [B,16,32,32]
        x = F.relu(self.conv2(x))     # [B,32,32,32]
        x = self.pool(x)              # [B,32,16,16]
        x = F.relu(self.conv3(x))     # [B,64,16,16]
        x = self.avgpool(x)           # [B,64,4,4]
        x = torch.flatten(x, 1)       # [B,64*4*4]
        x = self.fc(x)                # [B,10]
        return x

if __name__ == "__main__":
    # 1) Modeli oluştur
    num_classes = 10
    model = SimpleCNN(num_classes=num_classes)
    model.eval()

    # 2) Örnek giriş tensörü (dummy)
    example_input = torch.randn(1, 3, 64, 64)

    # 3) TorchScript trace
    traced = torch.jit.trace(model, example_input)

    # 4) Kaydet
    traced.save("model_ts.pt")
    print("TorchScript model kaydedildi: model_ts.pt")
```
Bu dosyayı çalıştırdığımda: 
* aynı klasörde model_ts.pt oluşur.
Sonra bu dosyayı C++ projemde exe’nin olduğu klasöre (ör: build/Debug) kopyalarım.

>Not: Bu model şu an random ağırlıklara sahip. Yani çıktılar anlamlı değil, ama pipeline tamamen gerçek.İleride eğittiğim ağırlıkları da aynı yapıyla export edip kullanacağım.

----
# Şimdi gelin .pt uzantılı dosyamızı oluşturalım.Yani aşağıdaki bu kodu çalıştırıp aynı klasör içerisine bu yapıyı entegre edelim.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes: int = 10):
        super().__init__()
        # [B, 3, 64, 64]
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)   # [B,16,64,64]
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # [B,32,64,64]
        self.pool = nn.MaxPool2d(2, 2)                            # boyut yarıya iner
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # [B,64,16,16] civarı
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))               # [B,64,4,4]
        self.fc = nn.Linear(64 * 4 * 4, num_classes)              # [B,10]

    def forward(self, x):
        x = F.relu(self.conv1(x))     # [B,16,64,64]
        x = self.pool(x)              # [B,16,32,32]
        x = F.relu(self.conv2(x))     # [B,32,32,32]
        x = self.pool(x)              # [B,32,16,16]
        x = F.relu(self.conv3(x))     # [B,64,16,16]
        x = self.avgpool(x)           # [B,64,4,4]
        x = torch.flatten(x, 1)       # [B,64*4*4]
        x = self.fc(x)                # [B,10]
        return x

if __name__ == "__main__":
    # 1) Modeli oluştur
    num_classes = 10
    model = SimpleCNN(num_classes=num_classes)
    model.eval()

    # 2) Örnek giriş tensörü (dummy)
    example_input = torch.randn(1, 3, 64, 64)

    # 3) TorchScript trace
    traced = torch.jit.trace(model, example_input)

    # 4) Kaydet
    traced.save("model_ts.pt")
    print("TorchScript model kaydedildi: model_ts.pt")

TorchScript model kaydedildi: model_ts.pt


---
## Şimdi ise C++ tarafını anlatıma geçiyorum
---

## 🔹 4.2 – C++ Tarafı: Resmi Modele Verip Forward Çalıştırma

Artık C++ tarafında:

* Resmi okuyup

* 64×64’e resize edip

* cv::Mat → torch::Tensor dönüşümü yapıp

* TorchScript CNN modeline input verip

* Çıktıyı yazdıracağım.

Aşağıdaki main.cpp yapısı bunu yapıyor:

```cpp
// SORU:
//  - TorchScript'e çevirdiğim CNN modelini C++ tarafında nasıl yüklerim?
//  - OpenCV'den gelen tensor_image'i modele input verip forward nasıl çalıştırırım?

#include <iostream>
#include <torch/script.h>
#include <opencv2/opencv.hpp>

int main() {
    try {
        std::cout << "[INFO] Program basladi.\n";

        // 1) TorchScript CNN modelini yükle
        const std::string model_path = "model_ts.pt";
        torch::jit::script::Module module = torch::jit::load(model_path);
        std::cout << "[OK] Model yüklendi: " << model_path << std::endl;

        torch::Device device(torch::kCPU);
        module.to(device);
        module.eval();

        // 2) Resmi oku
        const std::string image_path = "test.jpg";
        cv::Mat img = cv::imread(image_path);
        if (img.empty()) {
            std::cerr << "[HATA] Resim yuklenemedi: " << image_path << std::endl;
            std::cout << "Cikmak icin Enter'a bas...\n";
            std::cin.get();
            return -1;
        }

        std::cout << "[OK] Orijinal resim boyutu: "
                  << img.cols << "x" << img.rows << " (W x H)" << std::endl;

        // 3) Model 64x64 beklediği için resize
        cv::resize(img, img, cv::Size(64, 64));

        // 4) BGR -> RGB
        cv::cvtColor(img, img, cv::COLOR_BGR2RGB);

        // 5) uint8 -> float32 ve 0–1 araligina ölçekleme
        cv::Mat img_float;
        img.convertTo(img_float, CV_32F, 1.0 / 255.0);

        // 6) HWC -> tensor (N, H, W, C)
        auto tensor_image = torch::from_blob(
            img_float.data,
            {1, img_float.rows, img_float.cols, 3},        // [1, 64, 64, 3]
            torch::TensorOptions().dtype(torch::kFloat32).device(device)
        );

        // 7) NHWC -> NCHW
        tensor_image = tensor_image.permute({0, 3, 1, 2}).contiguous();
        tensor_image = tensor_image.clone();  // OpenCV belleginden bagimsiz kopya

        std::cout << "[OK] Input tensor shape (NCHW): " << tensor_image.sizes() << std::endl;

        // 8) Forward için input listesi hazırla
        std::vector<torch::jit::IValue> inputs;
        inputs.push_back(tensor_image);

        // 9) Forward
        torch::Tensor output = module.forward(inputs).toTensor();
        std::cout << "[OK] Forward bitti.\n";
        std::cout << "Output shape: " << output.sizes() << std::endl;

        // 10) Softmax + en yüksek olasılık sınıfını bul
        torch::Tensor probs = torch::softmax(output, 1);
        torch::Tensor pred_class = probs.argmax(1);

        std::cout << "Output (raw logits): " << output << std::endl;
        std::cout << "Output (probs): " << probs << std::endl;
        std::cout << "Predicted class index: " << pred_class.item<int>() << std::endl;

        std::cout << "Program bitti. Cikmak icin Enter'a bas...\n";
        std::cin.get();
    }
    catch (const c10::Error& e) {
        std::cerr << "[EXCEPTION - c10] " << e.what() << std::endl;
        std::cout << "Cikmak icin Enter'a bas...\n";
        std::cin.get();
        return -1;
    }
    catch (const std::exception& e) {
        std::cerr << "[EXCEPTION - std] " << e.what() << std::endl;
        std::cout << "Cikmak icin Enter'a bas...\n";
        std::cin.get();
        return -1;
    }

    return 0;
}


---

## 🔹 4.3 – Çalıştırma Adımları

* Python tarafında TorchScript modeli oluştur:


* Oluşan model_ts.pt dosyasını C++ projemde build/Debug içine kopyala:

```bash
Uygulama - 2/
    CMakeLists.txt
    main.cpp
    build/
        Debug/
            model_test.exe
            model_ts.pt
            test.jpg
            (DLL'ler)
```


C++ tarafında yeniden derle (main.cpp değiştiği için):
```bash
cd build
cmake ..
cmake --build .
cd Debug
model_test.exe
```


Beklediğim çıktı örneği:

* Input tensor shape: [1, 3, 64, 64]

* Output shape: [1, 10]

* Raw logits (10 değer)

* Softmax sonrası olasılıklar

* Predicted class index (0–9 arası bir integer)

**Model eğitilmediği için bu sınıf index’i anlamsız, ama deployment pipeline’ı tam çalışıyor.**

---

## 🔹 4.4 – Sonraki Adım: Eğitim + Gerçek Deployment

### Bu aşamadan sonra yapılacaklar:

#### Python tarafında:

* Aynı SimpleCNN mimarisiyle

* Gerçek bir dataset üzerinde eğitim (train/val loop)

- Eğitilen ağırlıkları TorchScript’e export etme (yine model_ts.pt)

#### C++ tarafında:

* Aynı C++ kodu (hiç bozmadan) kullanarak

* Eğitimli modeli load etmek

* Gerçek görüntü üzerinde anlamlı sınıflandırma yapmak

* İstersem bu modeli daha büyük bir pipeline’a (kamera, bounding box, UI) entegre etmek

Bu yapı ile:

**Eğitim → Python + PyTorch**

**Deployment → C++ + LibTorch + OpenCV**

**dünyasını net bir şekilde ayırmış oluyorum.**

---


---

Şimdi adımlar sende:

1. `export_cnn_ts.py`’yi çalıştır → `model_ts.pt` üret.  
2. `main.cpp`’yi bu CNN forward versiyonuyla değiştir.  
3. `cmake ..` + `cmake --build .` → `model_test.exe`’yi çalıştır.  
🚀  

---
---